In [2]:
import pandas as pd
import cv2
import mediapipe as mp
import os
# classes=["open palm","fist","index","left","right","up","down","OK","middle finger"]

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [4]:
def create_dataset(root="dataset", image_size=(224,224)):
    # Create folders
    img_folder = os.path.join(root, "images")
    os.makedirs(img_folder, exist_ok=True)

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    label = input("\nEnter the gesture name: ")

    datapoints = []
    data_count = 1

    print(f'\nInitiating Capture for {label}')

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't capture frame")
            continue

        h, w, _ = frame.shape
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_=frame.copy()
        result = hands.process(rgb)


        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

                # Extract coordinates
                x_list = [int(lm.x * w) for lm in hand_landmarks.landmark]
                y_list = [int(lm.y * h) for lm in hand_landmarks.landmark]

                # Bounding box for cropping
                x_min, x_max = max(min(x_list) - 20, 0), min(max(x_list) + 20, w)
                y_min, y_max = max(min(y_list) - 20, 0), min(max(y_list) + 20, h)

                hand_img = frame_[y_min:y_max, x_min:x_max]
                if hand_img.size == 0:  # skip empty crops
                    continue
                hand_img = cv2.resize(hand_img, (256,256))

                # Landmark dict
                landmarks = {"label": label, "image_path": ""}
                for i, lm in enumerate(hand_landmarks.landmark):
                    landmarks[f"x{i}"] = lm.x
                    landmarks[f"y{i}"] = lm.y
                    landmarks[f"z{i}"] = lm.z

                cv2.imshow("frame", frame)

                key = cv2.waitKey(1) & 0xFF
                if key == ord("q"):
                    cap.release()
                    cv2.destroyAllWindows()
                    return datapoints  # exit and return collected data

                elif key == ord("s"):  # save image + landmarks
                    img_name = f"{label}_{data_count}.jpg"
                    img_path = os.path.join(img_folder, img_name)
                    cv2.imwrite(img_path, hand_img)

                    landmarks["image_path"] = img_path
                    datapoints.append(landmarks)

                    print(f"{data_count}) {landmarks['label']} -> {img_path}")
                    data_count += 1

                elif key == ord("n"):  # switch label
                    label = input("\nEnter the Gesture Name: ")
                    print(f"\nInitiating Capture for {label}")
                    data_count = 1

    cap.release()
    cv2.destroyAllWindows()
    return datapoints



In [5]:
data = create_dataset()



Initiating Capture for open palm
1) open palm -> dataset\images\open palm_1.jpg
2) open palm -> dataset\images\open palm_2.jpg
3) open palm -> dataset\images\open palm_3.jpg
4) open palm -> dataset\images\open palm_4.jpg
5) open palm -> dataset\images\open palm_5.jpg
6) open palm -> dataset\images\open palm_6.jpg
7) open palm -> dataset\images\open palm_7.jpg
8) open palm -> dataset\images\open palm_8.jpg
9) open palm -> dataset\images\open palm_9.jpg
10) open palm -> dataset\images\open palm_10.jpg
11) open palm -> dataset\images\open palm_11.jpg
12) open palm -> dataset\images\open palm_12.jpg
13) open palm -> dataset\images\open palm_13.jpg
14) open palm -> dataset\images\open palm_14.jpg
15) open palm -> dataset\images\open palm_15.jpg
16) open palm -> dataset\images\open palm_16.jpg
17) open palm -> dataset\images\open palm_17.jpg
18) open palm -> dataset\images\open palm_18.jpg
19) open palm -> dataset\images\open palm_19.jpg
20) open palm -> dataset\images\open palm_20.jpg
21) 

In [7]:
df = pd.DataFrame(data)
df

,label,image_path,x0,y0,z0,x1,y1,z1,x2,y2,...,z17,x18,y18,z18,x19,y19,z19,x20,y20,z20
0,open palm,dataset\images\open palm_1.jpg,0.256909,0.674479,5.446344e-07,0.315704,0.629290,-0.033463,0.361203,0.538470,...,-0.033056,0.168942,0.394263,-0.050735,0.157012,0.351300,-0.057290,0.148280,0.307641,-0.060667
1,open palm,dataset\images\open palm_2.jpg,0.267407,0.668640,5.630786e-07,0.327819,0.626409,-0.035874,0.373232,0.538166,...,-0.034811,0.180098,0.390020,-0.053411,0.168077,0.346109,-0.060224,0.159705,0.302423,-0.063535
2,open palm,dataset\images\open palm_3.jpg,0.278813,0.665061,5.856877e-07,0.338139,0.624086,-0.033628,0.384026,0.537519,...,-0.035512,0.188901,0.387094,-0.053817,0.175854,0.341730,-0.061083,0.167391,0.296090,-0.064785
3,open palm,dataset\images\open palm_4.jpg,0.284100,0.651824,5.974576e-07,0.344148,0.615036,-0.036292,0.392779,0.532755,...,-0.036450,0.199438,0.378956,-0.055679,0.186524,0.333490,-0.063011,0.177936,0.287704,-0.066614
4,open palm,dataset\images\open palm_5.jpg,0.285429,0.644625,6.083113e-07,0.345301,0.609107,-0.038600,0.393774,0.527432,...,-0.035727,0.201354,0.375278,-0.055390,0.188593,0.330486,-0.063157,0.180218,0.285168,-0.067056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,middle finger,dataset\images\middle finger_17.jpg,0.729420,0.661799,1.412527e-07,0.684081,0.601694,0.011997,0.674080,0.538140,...,-0.005894,0.771726,0.486971,0.018098,0.758918,0.511260,0.039310,0.758569,0.534982,0.052338
189,middle finger,dataset\images\middle finger_18.jpg,0.702850,0.657644,1.340335e-07,0.657643,0.601373,0.006718,0.645663,0.537834,...,-0.005370,0.738647,0.487162,0.013064,0.726692,0.508195,0.030726,0.727325,0.530541,0.042242
190,middle finger,dataset\images\middle finger_19.jpg,0.161797,0.863263,-6.618782e-08,0.197193,0.819817,-0.029053,0.219182,0.754837,...,0.003907,0.149064,0.713285,-0.014146,0.155873,0.748152,-0.013939,0.160792,0.769280,-0.007212
191,middle finger,dataset\images\middle finger_20.jpg,0.166493,0.864761,-5.039930e-08,0.203332,0.820583,-0.030278,0.225588,0.754017,...,0.007146,0.152741,0.714173,-0.009990,0.159941,0.748001,-0.009461,0.163332,0.768437,-0.002697


In [8]:
df.shape


(193, 65)

In [10]:
df.to_csv('hand_gestures.csv', index=False)